In [1]:
# Import Dependencies
import pandas as pd
# import json

In [2]:
movies_csv = "movies_metadata.csv"
credits_csv = "credits.csv"

In [23]:
credits_df = pd.read_csv(credits_csv,low_memory=False)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
5,"[{'cast_id': 25, 'character': 'Lt. Vincent Han...","[{'credit_id': '52fe4292c3a36847f802916d', 'de...",949
6,"[{'cast_id': 1, 'character': 'Linus Larrabee',...","[{'credit_id': '52fe44959251416c75039da9', 'de...",11860
7,"[{'cast_id': 2, 'character': 'Tom Sawyer', 'cr...","[{'credit_id': '52fe46bdc3a36847f810f797', 'de...",45325
8,"[{'cast_id': 1, 'character': 'Darren Francis T...","[{'credit_id': '52fe44dbc3a36847f80ae0f1', 'de...",9091
9,"[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{'credit_id': '52fe426ec3a36847f801e14b', 'de...",710


In [37]:
movie_id = []
director = []

for idx, rows in credits_df.iterrows():
    crew_dict = eval(rows['crew'])
    movie_id.append(rows.id)
    try:
        director.append(crew_dict[0]['name'])
    except:
        director.append("n/a")
movie_director_df = pd.DataFrame({"id": movie_id, "Director": director})
movie_director_df.count()

id          45476
Director    45476
dtype: int64

In [68]:
movies_df = pd.read_csv(movies_csv,low_memory=False)
movies_dataprep = movies_df[['genres', 'budget', 'id', 'original_title', 'popularity', 
                            'production_companies', 'release_date', 'revenue']].copy()
movies_dataprep.head()
# remove rows with junk movie ids
movies_dataprep = movies_dataprep[movies_dataprep.id != '8/20/1997']
movies_dataprep = movies_dataprep[movies_dataprep.id != '9/29/2012']
movies_dataprep = movies_dataprep[movies_dataprep.id != '1/1/2014']

# typecast to int for merging
movies_dataprep.id = movies_dataprep.id.astype('int')
movie_director_df.id = movie_director_df.id.astype('int')

In [70]:
movies_with_director = pd.merge(movies_dataprep, movie_director_df, on = "id", how="left")
movies_with_director.head()

,genres,budget,id,original_title,popularity,production_companies,release_date,revenue,Director
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",30000000,862,Toy Story,21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]",10/30/1995,373554033.0,John Lasseter
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",65000000,8844,Jumanji,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",12/15/1995,262797249.0,Larry J. Franco
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",0,15602,Grumpier Old Men,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...",12/22/1995,0.0,Howard Deutch
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",16000000,31357,Waiting to Exhale,3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,12/22/1995,81452156.0,Forest Whitaker
4,"[{'id': 35, 'name': 'Comedy'}]",0,11862,Father of the Bride Part II,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...",2/10/1995,76578911.0,Alan Silvestri


In [71]:
movies_with_director.budget = pd.to_numeric(movies_with_director.budget, errors='coerce')
movies_with_director.revenue = pd.to_numeric(movies_with_director.revenue, errors='coerce')

In [72]:
#filter for budget >= 5000 and revenue >= 5000

movies_with_director = movies_with_director[(movies_with_director.budget >= 5000)]
movies_with_director = movies_with_director[(movies_with_director.revenue >= 5000)]

In [74]:
# add ROI column
movies_with_director['ROI'] = 100 * ((movies_with_director.revenue - movies_with_director.budget) 
                                     / movies_with_director.budget)
movies_with_director.head()

,genres,budget,id,original_title,popularity,production_companies,release_date,revenue,Director,ROI
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",30000000,862,Toy Story,21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]",10/30/1995,373554033.0,John Lasseter,1145.180110
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",65000000,8844,Jumanji,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",12/15/1995,262797249.0,Larry J. Franco,304.303460
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",16000000,31357,Waiting to Exhale,3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,12/22/1995,81452156.0,Forest Whitaker,409.075975
5,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",60000000,949,Heat,17.924927,"[{'name': 'Regency Enterprises', 'id': 508}, {...",12/15/1995,187436818.0,Michael Mann,212.394697
8,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",35000000,9091,Sudden Death,5.23158,"[{'name': 'Universal Pictures', 'id': 33}, {'n...",12/22/1995,64350171.0,Peter Hyams,83.857631


In [75]:
budget = []
genres = []
movie_id = []
orig_title = []
popularity = []
release_date = []
revenue = []
ROI = []
director = []

for idx, rows in movies_with_director.iterrows():
# orig_title = original_title
    genres_dict = eval(rows['genres'])
    for i in range(len(genres_dict)):
        budget.append(rows.budget)
        movie_id.append(rows.id)
        orig_title.append(rows.original_title)
        popularity.append(rows.popularity)
        release_date.append(rows.release_date)
        revenue.append(rows.revenue)
        ROI.append(rows.ROI)
        director.append(rows.Director)
        try:
            genres.append(genres_dict[i]['name'])
        except:
#             print(rows)
            genres.append("n/a")
            break
    
movies_final = pd.DataFrame({"genres": genres, "movie_id": movie_id, "budget": budget, 
                                     "original_title": orig_title, "popularity": popularity, 
                                     "release_date": release_date, "revenue":revenue, 
                                     "ROI":ROI, "Director": director})
movies_final.head()

,genres,movie_id,budget,original_title,popularity,release_date,revenue,ROI,Director
0,Animation,862,30000000,Toy Story,21.946943,10/30/1995,373554033.0,1145.18011,John Lasseter
1,Comedy,862,30000000,Toy Story,21.946943,10/30/1995,373554033.0,1145.18011,John Lasseter
2,Family,862,30000000,Toy Story,21.946943,10/30/1995,373554033.0,1145.18011,John Lasseter
3,Adventure,8844,65000000,Jumanji,17.015539,12/15/1995,262797249.0,304.30346,Larry J. Franco
4,Fantasy,8844,65000000,Jumanji,17.015539,12/15/1995,262797249.0,304.30346,Larry J. Franco


In [77]:
movies_final.to_csv('movies_final.csv', index=False)